In [1]:
import sqlite3
import pandas as pd
import geopandas as gpd
import os
import matplotlib.pyplot as plt
import numpy 
city_cube  = 'C_urban_cube_sh.sqlite'

C:\Users\bbel\AppData\Local\Programs\Python\Python39\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.10.1-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(


In [2]:
# get Eurostat number of population data
con = sqlite3.connect(city_cube)
# read full table
df = pd.read_sql_query("SELECT * FROM c_urban_cube_eurostat", con)

## Functions

### The first available year given a city/attribute

In [3]:
def first_year(df, city, attribute):
    line = df[(df['urau_code'] == city) & (df['indic_code'] == attribute)]
    if(len(line)==0):
        return None
    first_year = 1991
    col_set = line.columns
    col_set = col_set.drop(['index', 'indic_code', 'urau_code'])
    for col in col_set:
        if  not pd.isnull(line.iloc[0][col]): 
            first_year = col
            break
    return first_year

### Check if the data is available every 'sequence' year

In [10]:
def isAvailable(df, city, attribute, sequence):
    line = df[(df['urau_code'] == city) & (df['indic_code'] == attribute)]
    if(len(line)==0):
        return False
    year = first_year(df, city, attribute)
    col_set = df.columns
    for col in col_set:
        if col == year:
            break
        col_set = col_set.drop(col)
    available = True
    itterate = col_set
    for i in range(sequence):
        itterate = itterate[:-1]
    for idx, col in enumerate(itterate):
        if pd.isnull(line.iloc[0][col_set[idx+sequence]]):
            available = False
            break
    return available

### Number of available years of a city/attribute

In [11]:
def number_years(df, city, attribute):
    line = df[(df['urau_code'] == city) & (df['indic_code'] == attribute)]
    if(len(line)==0):
        return 0
    return len(line.columns) - line.isna().sum().sum() - 3

## Create an output dataset (city|attribute|number of available years|first available year|booleans on sequences)

In [16]:
%%time
d = {}
lcities = []
lattributes = []
firstyears = []
n_years = []
for c in df.urau_code.unique():
    for a in df.indic_code.unique():
        lcities.append(c)
        lattributes.append(a)
        firstyears.append(first_year(df, c, a))
        n_years.append(number_years(df, c, a))

d['city'] = lcities
d['attribute'] = lattributes
d['first_year'] = firstyears
d['n_years'] = n_years
seq_max = 3
for i in range(1, seq_max+1):
    lsequence = []
    for c in df.urau_code.unique():
        for a in df.indic_code.unique():
            lsequence.append(isAvailable(df, c, a, i))
    d['seq_'+ str(i)] = lsequence

df_result = pd.DataFrame(data=d)
df.to_csv('Available_seq.csv', index=False)
df_result.head()

CPU times: total: 1h 8min 3s
Wall time: 1h 9min 37s


,city,attribute,first_year,n_years,seq_1,seq_2,seq_3
0,AT001C,EN1002V,2004,1,False,False,False
1,AT001C,EN1003V,2004,2,False,False,False
2,AT001C,EN1004V,2004,2,False,False,False
3,AT001C,EN1005V,2004,2,False,False,False
4,AT001C,EN2002V,1992,22,False,False,False
